# Working with open data

## Libraries

In [1]:
import requests
from bs4 import BeautifulSoup

## Website of the city

In [2]:
from urllib.parse import urlencode
entrypoint = 'https://ru.wikipedia.org/w/index.php?'


def mkurl(title, oldid):
    return entrypoint + urlencode(dict(title=title, oldid=oldid))


def city_site(name):
    """
    Takes: name of the city.
    Returns: link to its official website from Russian Wikipedia. If there is no link -- None. 
    """
    url = mkurl(name, '')
    request = requests.get(url)
    if request.ok:
        page = BeautifulSoup(request.text, 'html.parser')
        info = page.find('span', attrs={'data-wikidata-property-id': 'P856'})
        if info:
            link = info('a')
            return link[0]['href']
    
    return None

In [3]:
res = [
   ('Москва', 'https://www.mos.ru/'),
   ('Абакан', 'http://абакан.рф/'), 
   ('Анадырь', 'https://novomariinsk.ru/'), 
   ('Киров (Кировская область)', 'http://www.mo-kirov.ru'), 
   ('Южно-Сахалинск', 'http://yuzhno-sakh.ru/'), 
   ('Усть-Каменоустюгск', None)
]

for city, site in res:
    assert city_site(city) == site

## Finding weather

In [4]:
def coordinates(place):
    """
    Takes: name of the city.
    Returns: coordinates of this city (latitude, longitude)
    """
    url = 'https://geocode-maps.yandex.ru/1.x/?geocode=' + place
    params = {
        'action': 'query',
        'format': 'json'
    }

    get_url = requests.get(url, params=params)
    info = get_url.json()

    raw_coor = info['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos']

    return float(raw_coor.split()[1]), float(raw_coor.split()[0])


def coordinates_as_string(place):
    x, y = coordinates(place)
    return str(x) + ', ' + str(y)


import time
import datetime


def show_weather(city, year, month, day):
    """
    Takes: city, year, month, day.
    Returns: prints the description of the weather in this city on this date.
    """
    
    date = datetime.date(year, month, day)
    unixtime = str(int(time.mktime(date.timetuple())))

    coors = coordinates_as_string(city)

    location = coors + ', ' + unixtime
    url = 'https://api.darksky.net/forecast/c4663f90dec1f746f0f200b8a3aea5b0/' + location
    params = {'units': 'si'}

    get_url = requests.get(url, params=params)
    query = get_url.json()

    # average temperature during the day (from 10 to 18)
    t1 = 0
    for hour in range(10, 18):
        t1 += query['hourly']['data'][hour]['apparentTemperature']
    t1 = round(t1 / 8)

    # average temperature during the night (from 00 to 7am)
    t2 = 0
    for hour in range(6):
        t2 += query['hourly']['data'][hour]['apparentTemperature']
    t2 = round(t2 / 6)

    date_str = str(day) + '/' + str(month) + '/' + str(year)

    message_weather = ('Here is information about the weather in {0} on {1}: \n\n'.format(city, date_str)
                       + 'Mean temperature during the day (in degrees Celsius): {}\n'.format(t1)
                       + 'Mean temperature at night (in degrees Celsius): {}\n'.format(t2)
                       + 'Weather summary: \n in the morning {0},\n during the day {1}, \n in the evening {2}.'.format(
                                                                        query['hourly']['data'][9]['summary'],
                                                                        query['hourly']['data'][15]['summary'],
                                                                        query['hourly']['data'][19]['summary'])
    )
    print(message_weather)

In [5]:
coordinates_as_string("Moscow")

'55.755814, 37.617635'

In [6]:
coordinates("Antananarivu")

(-18.901408, 47.522172)

In [7]:
# city
city = 'Moscow'

# date
year = 2018
month = 4
day = 1

show_weather(city, year, month, day)
print('\n\nMeanwhile in Antananarivu... \n\n')
show_weather('Antananarivu', year, month, day)

Here is information about the weather in Moscow on 1/4/2018: 

Mean temperature during the day (in degrees Celsius): -1
Mean temperature at night (in degrees Celsius): -2
Weather summary: 
 in the morning Overcast,
 during the day Overcast, 
 in the evening Mostly Cloudy.


Meanwhile in Antananarivu... 


Here is information about the weather in Antananarivu on 1/4/2018: 

Mean temperature during the day (in degrees Celsius): 23
Mean temperature at night (in degrees Celsius): 17
Weather summary: 
 in the morning Mostly Cloudy,
 during the day Mostly Cloudy, 
 in the evening Clear.
